# Spatially resolved depletion approaches

## One spatial dimension, uniform velocity profile, loosely coupled depletion
For each nuclide $i$:

$$\frac{\partial N_i(z, t)}{\partial t} + \nu_i(z, t) \frac{\partial N_i(z, t)}{\partial z} = S_i(z, t) - \mu_i(z, t) N_i(z, t)$$

where $N$ is the atom density, $t$ is time, $\nu$ is the linear flow rate, $z$ is the linear distance travelled, $S$ is the production, or source, term, and $\mu$ is the loss term.



### MORTY PDE

In order to reduce computational cost, the MORTY PDE approach looks only at isobars and without the advection loosely coupled to depletion.
The loose depletion coupling means that the terms in the equation can be updated as frequently or infrequently as desired. The problem can be run even without a depletion step (though it may be easier to run with depletion).
OpenMC can generate and output the relevant production and loss rates after a depletion step, which can then be fed into the MORTY PDE solver.

The solver uses upwind in space (backwards) and explicit in time finite differencing to generate a solution. At higher dimensions, a finite element approach would be better. The following equations show the finite difference approach.

$$\frac{\partial N_i(z, t)}{\partial t} + \nu_i(z, t) \frac{\partial N_i(z, t)}{\partial z} = S_i(z, t) - \mu_i(z, t) N_i(z, t)$$

Removing $i$ subscript for readability and applying finite differencing in space ($k$) and time ($l$).

$$\frac{N_{k, l+1} - N_{k, l}}{\Delta t} + \nu_{k, l} \frac{N_{k, l} - N_{k-1, l}}{\Delta z} = S_{k, l} - \mu_{k, l} N_{k, l}$$


$$N_{k, l+1} = N_{k, l} + \Delta t \left( S_{k, l} - \mu_{k, l} N_{k, l} - \nu_{k, l} \frac{N_{k, l} - N_{k-1, l}}{\Delta z} \right)$$



MORTY PDE runs this iteratively each time step five times by first updating the source terms for each nuclide in the isobar (since they decay into each other) and then calculating the concentration at that time step.

The level of fidelity of the source and loss terms can vary depending on the fidelity of the neutronics solve. At the lowest level, two regions can be used: in-core (in the neutronics solve) and ex-core (decay only).

## One spatial dimension, uniform velocity profile, tightly coupled predictor depletion

The general equation used in depletion calculations is [1]:

$$\frac{dn}{dt} = An$$

Where $n$ is a vector of nuclide concentrations (~2000-by-1) and $A$ is a square depletion matrix (~2000-by-2000) containing the source and loss terms for each nuclide.

Assuming that $A$ is constant over the time step $h$ (the "predictor" depletion method), Euler's method can be used:

$$n_{l+1} = n_l e^{A_l h}$$

Including a single spatial dimension with advection changes the depletion equation to:

$$\frac{\partial n}{\partial t} + \nu \frac{\partial n}{\partial z} = An$$

Applying finite differencing spatially:

$$\frac{\partial n_k}{\partial t} = A_k n_k - \frac{\nu}{\Delta z} (n_{k-1} - n_k)$$

Grouping $n_k$ together:

$$\frac{\partial n_k}{\partial t} = \left( A_k - \frac{\nu}{\Delta z} \right) n_k + \frac{\nu}{\Delta z} n_{k-1}$$

Generating the advective depletion matrix $\hat{A}$:

$$\hat{A}_k = A_k - \frac{\nu}{\Delta z}$$

$$\frac{\partial n_k}{\partial t} = \hat{A}_k n_k + \frac{\nu}{\Delta z} n_{k-1}$$

Applying Euler method:

$$n_{k, l+1} = n_{k, l} e^{\hat{A}_{k, l} h} + \frac{\nu_{k, l}}{\Delta z} n_{k-1, l}$$

### Approaches

#### Transfer Rates Method

This method takes the "transfer rates" approach from OpenMC for reprocessing and modifies it for advection use.
This approach is useful because implementation should be fairly rapid, and it will provide reasonably accurate results with a >3 materials included.
However, the approach requires many materials to have fine spatial resolution, and the cost scales rapidly as the matrix solve grows with the number of materials (simultaneous solve required).

An example with three materials (1, 2, and 3) is given below. Material 1 flows to material 2, 2 to 3, and 3 to 1.


$$
\frac{d}{dt}
\left(\begin{array}{c} 
n_1\\
n_2\\
n_3
\end{array}\right)
=
\left(\begin{array}{ccc} 
\hat{A}_{1, 1} & 0 & F_{3}\\
F_{1} & \hat{A}_{2, 2} & 0\\
0 & F_{2} & \hat{A}_{3, 3}
\end{array}\right)
\left(\begin{array}{c} 
n_1\\
n_2\\
n_3
\end{array}\right)
$$ 

Where:

$$F_k = \frac{\nu_k}{\Delta z}$$

$$\hat{A}_{mat, k} = A_{mat, k}- \frac{\nu_k}{\Delta z} I$$

This is *very* similar to transfer rates, as the $F_k$ terms are in units of per time, and can thus already be represented in the current version of OpenMC.
All that is missing is subtracting the diagonal advection term from the depletion matrices. If the flow rate and linear distance between materials is constant, then this only requires subtracting a constant scalar from the diagonal of each depletion matrix.



# References
[1] https://docs.openmc.org/en/stable/methods/depletion.html